In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from gensim.models import Word2Vec
from sklearn.manifold import TSNE 


Using TensorFlow backend.


# Visualizing Skills

In [2]:
#Load in skills dataframe
skill_df = pd.read_csv('skill.tsv', sep='\t').drop('Unnamed: 0', axis=1)
#Load in dictionary associating skill numbers with skill names
skill_dict = {}
with open('skill_dict.json', 'r', encoding='utf-8') as f:
    loaded = json.load(f)
    for v, k in loaded.items():
        skill_dict[k] = str(v) #Use number as key, string as value

In [3]:
#Read out the "sentences"
sentences=skill_df.iloc[:,1:].values.astype(str)
sentences=sentences.tolist()

In [5]:
#Each student is a "sentence", each skill is a "word"
#size = dimensionality of feature vectors
#window = max distance between current and predicted word within a sentence
#min_count = minimum number of occurrences within dataset
#workers = number of threads used
#sg = 0 (CBOW, default); = 1 (skip-gram)
model = Word2Vec(sentences, size=200, window=10, min_count=10, workers=4, sg=1, iter=100)

In [6]:
skill_num=model.wv.vocab; #Names of the words (numbers)
skill_vec=model[skill_num] #Access the vectors

skill_name=list()
#Associate with readable words
for k,v in skill_num.items(): #Iterate over the vocab from word2vec (k = key = number string)
    skill_name.append(skill_dict.get(k)) #Get the value (tag) saved at that key in the other dict
#print(skill_name)

In [7]:
tsne=TSNE(perplexity=30) #Instantiate the TSNE model (can change params here)
skill_tsne=tsne.fit_transform(skill_vec.astype(float)) #Run tsne

In [8]:
#Save as a tsv file for d3-scatterplot
# d={'x': skill_tsne[:,0],
#   'y': skill_tsne[:,1],
#   'skill' : skill_name}
tsne_save=pd.DataFrame({'x': skill_tsne[:,0],
  'y': skill_tsne[:,1],
  'skill' : skill_name})
tsne_save.to_csv('../d3-scatterplot/tsne_skills.tsv',sep='\t',index=False,columns=['x','y','skill'])

# Visualizing Assistments

In [75]:
# Load in Assistments ID dataframe
assistment_df = pd.read_csv('assistment_id.tsv', sep='\t').drop('Unnamed: 0', axis=1)
sentences=assistment_df.iloc[:,1:].values.astype(str)
sentences=sentences.tolist()

In [76]:
# Load entire Assistment dataframe to find the skills associated with each ID
filename = 'skill_builder_data_corrected.csv'
df = pd.read_csv(filename, encoding='ISO-8859-1', low_memory=False)
df = df[(df['original'] == 1) & (df['attempt_count'] == 1) & ~(df['skill_name'].isnull())]

In [97]:
#Each student is a "sentence", each skill is a "word"
#size = dimensionality of feature vectors
#window = max distance between current and predicted word within a sentence
#min_count = minimum number of occurrences within dataset
#workers = number of threads used
#sg = 0 (CBOW, default); = 1 (skip-gram)
model = Word2Vec(sentences, size=200, window=10, min_count=10, workers=4, sg=1, iter=30)

In [98]:
assist_num=model.wv.vocab; #Names of the words (numbers)
assist_vec=model[assist_num] #Access the vectors

tsne=TSNE(perplexity=30) #Instantiate the TSNE model (can change params here)
assist_tsne=tsne.fit_transform(assist_vec.astype(float)) #Run tsne

In [99]:
assist_skill=list()

for k,v in assist_num.items():
    skill = df[df['assistment_id'] == int(k)]['skill_name'].iloc[0] # get the first skill associated with the assistment
    assist_skill.append(skill)


In [100]:
#Save as a tsv file for d3-scatterplot
# d={'x': skill_tsne[:,0],
#   'y': skill_tsne[:,1],
#   'skill' : skill_name}
tsne_save=pd.DataFrame({'x': assist_tsne[:,0],
  'y': assist_tsne[:,1],
  'skill' : assist_skill})
tsne_save.to_csv('../d3-scatterplot/tsne_assist.tsv',sep='\t',index=False,columns=['x','y','skill'])